In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist
import re
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import *
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamroth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = os.getcwd()
path

'/Users/adamroth/Documents/flat_fe_work/mod4/week13/NLP---Twitter-Sentiment/notebooks/AR_notebooks'

### basic info

In [3]:
raw_data = pd.read_csv('../../data/judge-1377884607_tweet_product_company.csv', encoding = 'latin1')

In [4]:
raw_data.sample(15)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
1393,"÷¼ No fear, no envy, no meanness, no lies, no...",NaN,No emotion toward brand or product
4243,Hmm....only 9? #SXSW right-brain #mwrc11 left-...,iPad,Positive emotion
562,When will Google Circles launch? {link} &quot;...,NaN,No emotion toward brand or product
1429,SXSW 2011: Novelty of iPad news apps fades fas...,iPad or iPhone App,Negative emotion
2741,Brains are to zombies like content is to googl...,NaN,No emotion toward brand or product
2441,Google and Bing meet at #SXSW after their love...,Google,Negative emotion
8913,Time to switch to @mention on my iPhone so I c...,NaN,No emotion toward brand or product
3975,Great talk by @mention from @mention about gam...,NaN,No emotion toward brand or product
5155,RT @mention @mention Who said you won't be abl...,NaN,No emotion toward brand or product
2506,The forbidden apple has been spoiled! Long liv...,Apple,Negative emotion


In [5]:
raw_data['emotion_in_tweet_is_directed_at'].sample(10)

5191    Other Google product or service
2291                                NaN
4521                                NaN
1546                                NaN
6162                                NaN
3854                                NaN
3465                                NaN
2127                              Apple
5012                                NaN
3554                              Apple
Name: emotion_in_tweet_is_directed_at, dtype: object

In [6]:
raw_data['emotion_in_tweet_is_directed_at'].unique()

array(['iPhone', 'iPad or iPhone App', 'iPad', 'Google', nan, 'Android',
       'Apple', 'Android App', 'Other Google product or service',
       'Other Apple product or service'], dtype=object)

In [7]:
raw_data.isna()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
9088,False,False,False
9089,False,True,False
9090,False,True,False
9091,False,True,False


In [8]:
raw_data.dtypes

tweet_text                                            object
emotion_in_tweet_is_directed_at                       object
is_there_an_emotion_directed_at_a_brand_or_product    object
dtype: object

In [9]:
raw_data['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [10]:
#lots of nan's in this column, shouldn't matter while training model though

In [11]:
raw_data['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [12]:
#not many negatives, might need to mess with that, could use something like smote, might remove all from 'I cant tell'

### Tokenization

In [13]:
raw_data.iloc[i].tweet_text

NameError: name 'i' is not defined

In [13]:
raw_data.iloc[0].tweet_text.split()

['.@wesley83',
 'I',
 'have',
 'a',
 '3G',
 'iPhone.',
 'After',
 '3',
 'hrs',
 'tweeting',
 'at',
 '#RISE_Austin,',
 'it',
 'was',
 'dead!',
 'I',
 'need',
 'to',
 'upgrade.',
 'Plugin',
 'stations',
 'at',
 '#SXSW.']

In [14]:
test_list = raw_data.iloc[0].tweet_text.split()
type(test_list)

list

In [15]:
df = raw_data
df['split_text'] = 0
df.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product', 'split_text'],
      dtype='object')

In [16]:
raw_data['tweet_text'][5]

'@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conference http://ht.ly/49n4M #iear #edchat #asd'

In [17]:
for i in range(0,len(raw_data)):
    text = raw_data['tweet_text'][i]
    split_text = str(text).split()
    df['split_text'][i] = split_text
    
    



/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
df['split_text'][4]

['@sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 '#SXSW:',
 'Marissa',
 'Mayer',
 '(Google),',
 'Tim',
 "O'Reilly",
 '(tech',
 'books/conferences)',
 '&amp;',
 'Matt',
 'Mullenweg',
 '(Wordpress)']

In [19]:
import string
punctuation = string.punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [20]:
def clean_punct(string):

    clean_text_list = [''.join(ch for ch in s if ch not in punctuation) for s in string]
    
    #clean_text = ""
    
    #return (clean_text.join(clean_text_list))
    return clean_text_list
#cleans up punctuation

In [21]:
string = "@sxtxstate"
clean_punct(string)
print(clean_punct(string))

['', 's', 'x', 't', 'x', 's', 't', 'a', 't', 'e']


In [22]:
clean_punct(df['split_text'][4])

['sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 'SXSW',
 'Marissa',
 'Mayer',
 'Google',
 'Tim',
 'OReilly',
 'tech',
 'booksconferences',
 'amp',
 'Matt',
 'Mullenweg',
 'Wordpress']

In [23]:
# def clean_column(df['column']):
#     for i in range(0, len(df)):
#         thing = df['column'][i]
#         clean_string = str(thing)
#         return 

In [24]:
df['no_punct'] = 0

In [25]:
for i in range(0, len(df)):
    has_punct = df['split_text'][i]
    no_punct = clean_punct(has_punct)
    df['no_punct'][i] = no_punct

/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
df['no_punct'][1]

['jessedee',
 'Know',
 'about',
 'fludapp',
 '',
 'Awesome',
 'iPadiPhone',
 'app',
 'that',
 'youll',
 'likely',
 'appreciate',
 'for',
 'its',
 'design',
 'Also',
 'theyre',
 'giving',
 'free',
 'Ts',
 'at',
 'SXSW']

In [27]:
df['clean'] = 0
df['clean'].head()

0    0
1    0
2    0
3    0
4    0
Name: clean, dtype: int64

In [28]:
for i in range(0, len(df)):
    a = df['no_punct'][i]
    now_lower = str(a).lower()
    df['clean'][i] = now_lower
#I'm a dumb dumb who forgot that df.apply is a thing

/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [29]:
df['clean'].head()

0    ['wesley83', 'i', 'have', 'a', '3g', 'iphone',...
1    ['jessedee', 'know', 'about', 'fludapp', '', '...
2    ['swonderlin', 'can', 'not', 'wait', 'for', 'i...
3    ['sxsw', 'i', 'hope', 'this', 'years', 'festiv...
4    ['sxtxstate', 'great', 'stuff', 'on', 'fri', '...
Name: clean, dtype: object

In [30]:
print(len(df))
df.sample(10)

9093


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,split_text,no_punct,clean
8631,#SocialNetworks: #Google noch heute mit #Circl...,NaN,I can't tell,"[#SocialNetworks:, #Google, noch, heute, mit, ...","[SocialNetworks, Google, noch, heute, mit, Cir...","['socialnetworks', 'google', 'noch', 'heute', ..."
100,New buzz? &quot;@mention Google to Launch Majo...,NaN,No emotion toward brand or product,"[New, buzz?, &quot;@mention, Google, to, Launc...","[New, buzz, quotmention, Google, to, Launch, M...","['new', 'buzz', 'quotmention', 'google', 'to',..."
1010,Temporary Pop Shop by #Apple at #SXSW and Nice...,NaN,Positive emotion,"[Temporary, Pop, Shop, by, #Apple, at, #SXSW, ...","[Temporary, Pop, Shop, by, Apple, at, SXSW, an...","['temporary', 'pop', 'shop', 'by', 'apple', 'a..."
4502,Anyone find the makeshift #Apple store at #SXS...,Apple,Positive emotion,"[Anyone, find, the, makeshift, #Apple, store, ...","[Anyone, find, the, makeshift, Apple, store, a...","['anyone', 'find', 'the', 'makeshift', 'apple'..."
5427,"RT @mention Apple heads to SXSW, sets up tempo...",NaN,No emotion toward brand or product,"[RT, @mention, Apple, heads, to, SXSW,, sets, ...","[RT, mention, Apple, heads, to, SXSW, sets, up...","['rt', 'mention', 'apple', 'heads', 'to', 'sxs..."
1936,I just realized my iPhone's gonna go nuts with...,NaN,I can't tell,"[I, just, realized, my, iPhone's, gonna, go, n...","[I, just, realized, my, iPhones, gonna, go, nu...","['i', 'just', 'realized', 'my', 'iphones', 'go..."
8378,@mention tell @mention to give out gummy worms...,NaN,No emotion toward brand or product,"[@mention, tell, @mention, to, give, out, gumm...","[mention, tell, mention, to, give, out, gummy,...","['mention', 'tell', 'mention', 'to', 'give', '..."
8462,"&quot;If there's one thing SXSW is good for, i...",Apple,Positive emotion,"[&quot;If, there's, one, thing, SXSW, is, good...","[quotIf, theres, one, thing, SXSW, is, good, f...","['quotif', 'theres', 'one', 'thing', 'sxsw', '..."
5330,RT @mention #Want: Micro USB charger for Samsu...,NaN,No emotion toward brand or product,"[RT, @mention, #Want:, Micro, USB, charger, fo...","[RT, mention, Want, Micro, USB, charger, for, ...","['rt', 'mention', 'want', 'micro', 'usb', 'cha..."
5978,RT @mention Heard about Apple's pop-up store i...,NaN,No emotion toward brand or product,"[RT, @mention, Heard, about, Apple's, pop-up, ...","[RT, mention, Heard, about, Apples, popup, sto...","['rt', 'mention', 'heard', 'about', 'apples', ..."


In [37]:
sw = stopwords.words('english')

In [38]:
def doc_preparer(doc, stop_words=sw):
    '''
    
    :param doc: a document from the corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in stop_words]
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)


In [42]:
extra_sw = pd.read_csv('../asp_notebooks/apl_prd_custom_stop.csv')
extra_sw = list(extra_sw['tweet_text'])
extra_sw


['go',
 'open',
 'line',
 'one',
 'amp',
 'come',
 'win',
 'like',
 'via',
 'launch',
 'cool',
 'day',
 'time',
 'temporary',
 'use',
 'apps',
 'love',
 'downtown',
 'free',
 'great']

In [62]:
sw = sw + extra_sw
sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [56]:
# def remove_sw(df):
#     for i in range(0, len(df)):
#         for word in df['clean'][i]:
#             if word in sw:
#                 word.remove(sw)
        

            

In [70]:
df['clean'] = [' '.join([a for a in x.split() if a not in sw]) for x in df['clean']]
#shouldn't that work?
df['clean'].head(10)

0    ['wesley83', 'i', 'have', 'a', '3g', 'iphone',...
1    ['jessedee', 'know', 'about', 'fludapp', '', '...
2    ['swonderlin', 'can', 'not', 'wait', 'for', 'i...
3    ['sxsw', 'i', 'hope', 'this', 'years', 'festiv...
4    ['sxtxstate', 'great', 'stuff', 'on', 'fri', '...
5    ['teachntech00', 'new', 'ipad', 'apps', 'for',...
6                                              ['nan']
7    ['sxsw', 'is', 'just', 'starting', 'ctia', 'is...
8    ['beautifully', 'smart', 'and', 'simple', 'ide...
9    ['counting', 'down', 'the', 'days', 'to', 'sxs...
Name: clean, dtype: object

In [33]:
df_no_emo = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'No emotion toward brand or product']
print(len(df_no_emo))


5389


In [34]:
df_neg = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion']
len(df_neg)

570

In [35]:
df_pos = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion']
len(df_pos)

2978

In [36]:
df_unclear = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'I can\'t tell']
print(len(df_unclear))
df_unclear['clean'][:5]

156


90     ['thanks', 'to', 'mention', 'for', 'publishing...
102    ['\x89ûïmention', 'quotapple', 'has', 'opened'...
237    ['just', 'what', 'america', 'needs', 'rt', 'me...
341    ['the', 'queue', 'at', 'the', 'apple', 'store'...
368    ['hope', 'its', 'better', 'than', 'wave', 'rt'...
Name: clean, dtype: object

In [36]:
# X = df_pos.clean.astype('str')
# y = df.is_there_an_emotion_directed_at_a_brand_or_product

<WordListCorpusReader in '/Users/adamroth/nltk_data/corpora/stopwords'>